In [11]:
from math import log
import treePlotter as tpl
import operator
import csv

def createDataSet():
    with open('PlayTennis.csv', 'r') as f:
        reader = csv.reader(f,delimiter=',')
        dataSet = list(reader)
    return dataSet[1:], dataSet[0]

In [12]:
a,b=createDataSet()
print(a)

[['Sunny', 'Hot', 'High', 'Weak', 'No'], ['Sunny', 'Hot', 'High', 'Strong', 'No'], ['Overcast', 'Hot', 'High', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Weak', 'Yes'], ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'], ['Rain', 'Cool', 'Normal', 'Strong', 'No'], ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'], ['Sunny', 'Mild', 'High', 'Weak', 'No'], ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'], ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'], ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'], ['Overcast', 'Mild', 'High', 'Strong', 'Yes'], ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Strong', 'No']]


In [13]:
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet: # Le nombre d'occurrences pour chaque attribut
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2) # log base 2
    return shannonEnt

In [14]:
calcShannonEnt(a)

0.9402859586706309

In [15]:
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis] # Mettre dehors l'axe utilisé pour la subdivision
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [16]:
c = splitDataSet(a,0,'Overcast')
print(len(c),c)

4 [['Hot', 'High', 'Weak', 'Yes'], ['Cool', 'Normal', 'Strong', 'Yes'], ['Mild', 'High', 'Strong', 'Yes'], ['Hot', 'Normal', 'Weak', 'Yes']]


In [17]:
splitDataSet(c,2,'weak')

[]

In [18]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1 # Dernière colonne contient la classe d'appartenance
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures): # Pour chaque attribut
        featList = [example[i] for example in dataSet] # Liste des valeurs
        uniqueVals = set(featList) # Liste des valeurs sans doublons
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy # Calculer le gain
        if (infoGain > bestInfoGain): # Garder le meilleur gain
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature # rang de l'attribut: entier

In [19]:
ra = chooseBestFeatureToSplit(a)
print(ra)

0


In [20]:
rc=chooseBestFeatureToSplit(c)
print(rc)

-1


In [21]:
def majorityCnt(classList): # Retourner la classe avec la majorité de vote
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=lambda item: item[1], reverse=True)
    #print(sortedClassCount)
    return sortedClassCount[0][0]
majorityCnt(['no','no','no','yes'])

'no'

In [22]:
def createTree(dataSet,labels):
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0] # Arrêter la decomposition (Exemples de la même classe)
    if len(dataSet[0]) == 1:
        return majorityCnt(classList) # Arrêter s'il ne reste qu'un seul attribut
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    print(bestFeatLabel)
    myTree = {bestFeatLabel:{}} # Initialiser le dictionnaire
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels.copy() # Copier chaque fois les noms des attributs
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree

In [23]:
dataSet, labels=createDataSet()
tr=createTree(dataSet, labels)
print(tr)

Outlook
Humidity
Wind
{'Outlook': {'Sunny': {'Humidity': {'Normal': 'Yes', 'High': 'No'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}}}}
